In [7]:
from flask import Flask, request, jsonify, send_file
from werkzeug.utils import secure_filename
import os
import docx
import PyPDF2
import nltk

nltk.download('punkt')

app = Flask(__name__)

UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
ALLOWED_EXTENSIONS = {'docx', 'pdf'}

# Ensure the upload folder exists
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

# Utility function to check allowed file types
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Function to parse DOCX files
def parse_docx(filepath):
    doc = docx.Document(filepath)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# Function to parse PDF files
def parse_pdf(filepath):
    pdf_text = ""
    with open(filepath, 'rb') as pdf_file:
        reader = PyPDF2.PdfFileReader(pdf_file)
        for page_num in range(reader.numPages):
            page = reader.getPage(page_num)
            pdf_text += page.extractText()
    return pdf_text

# Function to extract keywords from a job description (simple keyword-based extraction)
def extract_keywords(job_description):
    # Example keyword extraction, can be expanded based on complexity needed
    words = nltk.word_tokenize(job_description)
    keywords = [word for word in words if word.isalnum()]
    return keywords

# Function to customize the resume (simple keyword matching)
def customize_resume(resume_text, job_keywords):
    customized_resume = []
    for line in resume_text.split('\n'):
        if any(keyword.lower() in line.lower() for keyword in job_keywords):
            customized_resume.append(line)
    return '\n'.join(customized_resume)

# Route to upload resume and job description, then generate customized resume
@app.route('/upload', methods=['POST'])
def upload_files():
    # Check if the POST request has the files
    if 'resume' not in request.files or 'job_description' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    resume_file = request.files['resume']
    job_description_file = request.files['job_description']

    if resume_file.filename == '' or job_description_file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if resume_file and allowed_file(resume_file.filename):
        resume_filename = secure_filename(resume_file.filename)
        resume_path = os.path.join(app.config['UPLOAD_FOLDER'], resume_filename)
        resume_file.save(resume_path)

        # Parse resume depending on the file type
        if resume_filename.endswith('.docx'):
            resume_text = parse_docx(resume_path)
        elif resume_filename.endswith('.pdf'):
            resume_text = parse_pdf(resume_path)
        else:
            return jsonify({'error': 'Unsupported file format'}), 400
    else:
        return jsonify({'error': 'Invalid resume file format'}), 400

    # Parse job description
    job_description_text = job_description_file.read().decode('utf-8')
    job_keywords = extract_keywords(job_description_text)

    # Customize resume based on job description keywords
    customized_resume_text = customize_resume(resume_text, job_keywords)

    # Save customized resume to a file
    customized_resume_path = os.path.join(app.config['UPLOAD_FOLDER'], 'customized_resume.txt')
    with open(customized_resume_path, 'w') as f:
        f.write(customized_resume_text)

    return send_file(customized_resume_path, as_attachment=True, download_name='customized_resume.txt')

if __name__ == '__main__':
    app.run(debug=True, port=5001)  # Change the port from 5000 to 5001 or any unused port



 * Serving Flask app '__main__'
 * Debug mode: on


[nltk_data] Downloading package punkt to /home/ash93/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/ash93/projects/resumefit/server/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/ash93/projects/resumefit/server/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/ash93/projects/resumefit/server/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/home/ash93/projects/resumefit/server/.venv/lib/pyt

SystemExit: 1

In [1]:
from flask import Flask, request, jsonify
import os

app = Flask(__name__)

# Directory where the uploaded files will be stored
UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
ALLOWED_EXTENSIONS = {'pdf', 'docx'}

# Helper function to check allowed file extensions
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Route for uploading the resume
@app.route('/upload_resume', methods=['POST'])
def upload_resume():
    if 'file' not in request.files:
        return jsonify({"error": "No file part in the request"}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected"}), 400
  
    if file and allowed_file(file.filename):
        filename = file.filename
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        return jsonify({"message": "Resume uploaded successfully", "file_path": file_path}), 200
    else:
        return jsonify({"error": "Invalid file type"}), 400

# Route for uploading the job description
@app.route('/upload_job_description', methods=['POST'])
def upload_job_description():
    if 'file' not in request.files:
        return jsonify({"error": "No file part in the request"}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected"}), 400

    if file and allowed_file(file.filename):
        filename = file.filename
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        return jsonify({"message": "Job description uploaded successfully", "file_path": file_path}), 200
    else:
        return jsonify({"error": "Invalid file type"}), 400

if __name__ == '__main__':
    # Running on a different port to avoid conflicts
    app.run(debug=True, port=5001, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [05/Oct/2024 12:01:17] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [05/Oct/2024 12:01:17] "GET /favicon.ico HTTP/1.1" 404 -


In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
import requests
from dotenv import load_dotenv, find_dotenv

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Load API key from environment variables
load_dotenv(find_dotenv())
api_key = os.getenv('GOOGLE_API_KEY')

# Directory where the uploaded files will be stored
UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
ALLOWED_EXTENSIONS = {'pdf', 'docx'}

# Helper function to check allowed file extensions
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def index():
    return 'Welcome to the Resume Customization Tool!'

@app.route('/upload_resume', methods=['POST'])
def upload_resume():
    if 'file' not in request.files:
        return jsonify({"error": "No file part in the request"}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected"}), 400

    if file and allowed_file(file.filename):
        filename = file.filename
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        return jsonify({"message": "Resume uploaded successfully", "file_path": file_path}), 200
    else:
        return jsonify({"error": "Invalid file type"}), 400

@app.route('/upload_job_description', methods=['POST'])
def upload_job_description():
    if 'file' not in request.files:
        return jsonify({"error": "No file part in the request"}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected"}), 400

    if file and allowed_file(file.filename):
        filename = file.filename
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        return jsonify({"message": "Job description uploaded successfully", "file_path": file_path}), 200
    else:
        return jsonify({"error": "Invalid file type"}), 400

@app.route('/generate_response', methods=['POST'])
def generate_response():
    user_query = request.json.get('user_query')

    if not user_query:
        return jsonify({"error": "No user query provided"}), 400

    # Set the prompt for the response
    prompt = f"Resume customization task:\nUser: {user_query}\nAI:"

    # Set the API endpoint
    endpoint = "https://generativelanguage.googleapis.com/v1beta"

    # Prepare the request body
    request_body = {
        "contents": [
            {
                "parts": [
                    {
                        "text": prompt
                    }
                ]
            }
        ]
    }

    # Make the request to the API
    response = requests.post(
        f"{endpoint}/models/gemini-pro:generateContent?key={api_key}",
        json=request_body,
    )

    # Check if the response is successful
    if response.status_code != 200:
        return jsonify({"error": "Failed to generate response from the API"}), 500

    # Parse the response
    response_json = response.json()
    
    # Check if the response contains expected data
    if "candidates" in response_json and len(response_json["candidates"]) > 0:
        generated_content = response_json["candidates"][0]['content']['parts'][0]['text']
        return jsonify({"generated_response": generated_content}), 200
    else:
        return jsonify({"error": "No valid response from the API"}), 500

if __name__ == '__main__':
    app.run(debug=True, port=5001)


ModuleNotFoundError: No module named 'flask_cors'

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
import requests
from dotenv import load_dotenv, find_dotenv
from werkzeug.utils import secure_filename
import docx
import PyPDF2

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Load API key from environment variables
load_dotenv(find_dotenv())
api_key = os.getenv('YOUR_API_KEY')  # Replace with your actual API key

# Directory where the uploaded files will be stored
UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
ALLOWED_EXTENSIONS = {'pdf', 'docx'}

# Helper function to check allowed file extensions
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Function to extract text from DOCX files
def extract_text_from_docx(filepath):
    doc = docx.Document(filepath)
    return '\n'.join([para.text for para in doc.paragraphs])

# Function to extract text from PDF files
def extract_text_from_pdf(filepath):
    text = ""
    with open(filepath, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

@app.route('/')
def index():
    return 'Welcome to the Resume Customization Tool!'

@app.route('/upload_resume', methods=['POST'])
def upload_resume():
    if 'file' not in request.files:
        return jsonify({"error": "No file part in the request"}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected"}), 400

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        
        # Extract text from the resume
        if filename.endswith('.docx'):
            resume_text = extract_text_from_docx(file_path)
        elif filename.endswith('.pdf'):
            resume_text = extract_text_from_pdf(file_path)

        return jsonify({"message": "Resume uploaded successfully", "resume_text": resume_text}), 200
    else:
        return jsonify({"error": "Invalid file type"}), 400

@app.route('/upload_job_description', methods=['POST'])
def upload_job_description():
    if 'file' not in request.files:
        return jsonify({"error": "No file part in the request"}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected"}), 400

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        
        # Extract text from the job description
        if filename.endswith('.docx'):
            job_desc_text = extract_text_from_docx(file_path)
        elif filename.endswith('.pdf'):
            job_desc_text = extract_text_from_pdf(file_path)

        return jsonify({"message": "Job description uploaded successfully", "job_description_text": job_desc_text}), 200
    else:
        return jsonify({"error": "Invalid file type"}), 400

@app.route('/generate_customized_resume', methods=['POST'])
def generate_customized_resume():
    resume_text = request.json.get('resume_text')
    job_description = request.json.get('job_description')

    if not resume_text or not job_description:
        return jsonify({"error": "Both resume text and job description are required"}), 400

    # Set the prompt for the API
    prompt = f"Customize the following resume based on this job description:\n\nJob Description: {job_description}\n\nResume: {resume_text}\n\nCustomized Resume:"

    # Set the API endpoint
    endpoint = "https://generativelanguage.googleapis.com/v1beta"

    # Prepare the request body
    request_body = {
        "contents": [
            {
                "parts": [
                    {
                        "text": prompt
                    }
                ]
            }
        ]
    }

    # Make the request to the API
    response = requests.post(
        f"{endpoint}/models/gemini-pro:generateContent?key={api_key}",
        json=request_body,
    )

    # Check if the response is successful
    if response.status_code != 200:
        return jsonify({"error": "Failed to generate response from the API"}), 500

    # Parse the response
    response_json = response.json()
    
    # Check if the response contains expected data
    if "candidates" in response_json and len(response_json["candidates"]) > 0:
        customized_resume = response_json["candidates"][0]['content']['parts'][0]['text']
        return jsonify({"customized_resume": customized_resume}), 200
    else:
        return jsonify({"error": "No valid response from the API"}), 500

if __name__ == '__main__':
    app.run(debug=True, port=5001)
